## Toy model 3:

<img src="ABC_toy_model_3.png" alt="Toy Model 3" width="800"/>

## Unit test for ABC toy model 3

Current model does have A-->B

True model doesn't have A-->B reaction

Conditions
1. observe growth with A in media but not B
2. observe growth with B in media but not A
3. observe no growth with A in media but not B - with a knockout for A-->C
4. observe growth with B in the media but not A - with a knockout for A-->C

### CROP problem formulation (not relaxed)

If we want a nogrowth condition where only $B$ is in the media, and not $A$, and we have a growth condition where $A$ is in the media, but not $B$, then we rewrite as below:

$$\begin{array}{lll}
\min_z &  weights^T(1 - z) \text{\ \ \ \ \ \ if you have to remove a reaction, choose one with lowest likelihood (weight) }\\ \\
 &\begin{array}{lll}
& S^Tm + e_{C\rightarrow} = r_{nogrowth} & \text{Gibbs free energy-like constraint. Dual version of steady-state constraint. All equality constraints come into this part.}\\ 
& r_{nogrowth,i}\leq\Omega\cdot(1-z_i) & \text{for $i\neq \rightarrow A$ or $\rightarrow B$ and the reaction is part of the model $z_{i}=1$ then the dual variable, $r$, is {\it{not}} allowed to be positive. If the reaction is not part of model, then $r$ is allowed to be positive.} \\
& r_{nogrowth,\rightarrow A} \geq 0 & \text{Dual variable $r$ is allowed to be positive (tight) for A import reaction.}\\
& r_{nogrowth,\rightarrow B} \geq 0 & \text{Dual variable $r$ is allowed to be positive (tight) for B import reaction.}\\ 
& U_{B_2}\cdot r_{nogrowth,\rightarrow B}   \leq U_{nogrowth} & \text{The solution to the dual will exactly equal the primal, when this condition is met. Upper bound for $r_{nogrowth,\rightarrow B}$ } \\
\end{array} \\
& Sv_{growth} = 0 \text{\ \ \ \ \ \ steady state constraint in growth condition. }\\
& 0\leq v_{growth,i} \leq U_{growth,i}\cdot z_i  \text{\ \ \ \ \ \ for $i\neq \rightarrow A$ or $\rightarrow B$. No flux is allowed if you aren't part of the model.}\\
& v_{growth,\rightarrow A} <= U_{growth,\rightarrow A} \text{\ \ \ \ \ \ upper bound for A import flux reaction}\\
& v_{growth,\rightarrow B} <= 0 \text{\ \ \ \ \ \ upper bound for B import flux reaction} \\
& v_{growth,C\rightarrow} \geq U_{growth,C\rightarrow} \text{\ \ \ \ \ \ minimal growth condition} \\
& z_{\rightarrow A} = z_{\rightarrow B} = 1 \text{\ \ \ \ \ \ both A and B import reactions are in the model} \\  % maybe not necessary
& z\in \{0,1\} \text{\ \ \ \ \ \ $Z_i=0$ means the reaction $i$ is not in the model, and $Z_i=1$ means the reaction $i$ is in the model}
\end{array}$$

where the constant $M$ balances
1. the model's consistency with reaction weights versus
2. consistency with growth phenotypes

and $ e_{C\rightarrow}$ is a unit vector which is all zeros except for the export flux for C. This selects for the flux of C (bio growth flux).

and $r_{nogrowth}$ is the dual variable associated with the flux bounds in the primal problem (regular FBA). 0 means the bounds are not tight, negative means the lower bound is tight, positive means the upper bound is tight. 

and $m$ is the dual variable associated with the steady state equality constraint. $m<0$ if relaxing lower bound on steady state would increase growth. $m > 0$ if relaxing the upper bound on steady state would increase growth.

and $\Omega$ is a large postive number

where $S$ is

|  | $A_{SRC}\rightarrow A_{int}$ | $A_{int}\rightarrow B_{int}$ | $B_{int}\rightarrow C_{int}$ | $C_{int}\rightarrow C_{SNK}$ |  $A_{int}\rightarrow C_{int}$  | $B_{SRC}\rightarrow B_{int}$  |
|----------|:-------------:|:-------------:|:-------------:|:-------------:|:-------------:|:-------------:|
| $A_{int}$ |+1 |-1 | 0 | 0 |-1 |0 |
| $B_{int}$ | 0 |+1 |-1 | 0 | 0 |+1|
| $C_{int}$ | 0 | 0 |+1 |-1 |+1 |0 |

In [1]:
import cobra as cb 
from crop import model_from_stoich_matrix, PhenotypeObservation
import pandas as pd


def expected_model():
    '''Generates the expected cobra model for the ABC toy model 3.'''
    S_index = ['A_int', 'B_int', 'C_int']
    S_dict = {'A_SRC->A_int': [1,0,0],
            'A_int->B_int': [-1,1,0],
            'B_int->C_int': [0,-1,1],
            'C_int->C_SNK': [0,0,-1],
            'A_int->C_int': [-1,0,1],
            'B_SRC->B_int': [0,1,0],
            }
    S_table = pd.DataFrame( S_dict ,index=S_index)
    mets, rxns = S_table.index, S_table.columns
    n_mets = len(mets)
    n_rxns = len(rxns)

    
    growth_objective = {rxn_id:1 if rxn_id=='C_int->C_SNK' else 0 for rxn_id in rxns}
    upper_flux_bounds = {rxn_id:0 for rxn_id in rxns}
    lower_flux_bounds = {rxn_id:0 for rxn_id in rxns}

    expected_model = model_from_stoich_matrix(S_table, 
                                              "ABC_toy_model_3", 
                                              obj=growth_objective,
                                              lower_flux_bounds=lower_flux_bounds,
                                              upper_flux_bounds=upper_flux_bounds
                                              )
    return expected_model

expected_model = expected_model()


cb.io.save_json_model(expected_model, "ABC_toy_model_3.json")




In [8]:
import optlang as op
from optlang import Variable
import pandas as pd


# check the constraints
x = Variable('x', )
y = Variable('y')
constraint = op.Constraint(x+y)
print(constraint.to_json())



{'name': '98106a0e-bc9b-11ee-b849-4ebc8cb1c028', 'expression': {'type': 'Add', 'args': [{'type': 'Symbol', 'name': 'x'}, {'type': 'Symbol', 'name': 'y'}]}, 'lb': None, 'ub': None, 'indicator_variable': None, 'active_when': 1}


In [5]:
cb.util.array.create_stoichiometric_matrix(expected_model, array_type='DataFrame')


,A_SRC->A_int,A_int->B_int,B_int->C_int,C_int->C_SNK,A_int->C_int,B_SRC->B_int
A_int,1.0,-1.0,0.0,0.0,-1.0,0.0
B_int,0.0,1.0,-1.0,0.0,0.0,1.0
C_int,0.0,0.0,1.0,-1.0,1.0,0.0
